In [1]:
import os
import sys
import numpy as np
import healpy as hp
import histlite as hl
import csky as cy
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
from matplotlib import cm
import matplotlib.colors as colors

import seaborn as sns
# sns.set_theme(style="whitegrid")

%matplotlib inline
# %matplotlib notebook

from glob import glob
timer = cy.timing.Timer()
time = timer.time

sys.path.append('../../')
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from greco_grb.scripts.utils import *


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [2]:
class args:
    grb_idx_start = 168 
    grb_idx_end = 174 
    ncpu = 4
    out_dir = "/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output"
# import argparse
# p = argparse.ArgumentParser(description="Unbindling script",
#                             formatter_class=argparse.RawTextHelpFormatter)
# p.add_argument("--grb_idx_start", default=0, type=int, help="GRB idx start: [start, end)")
# p.add_argument("--grb_idx_end", default=1, type=int, help="GRB idx end: [start, end)")
# p.add_argument("--ncpu", default=1, type=int, help="Number of CPUs")
# args = p.parse_args()

In [3]:
print("\n===== Setting up csky =====\n")
data_dir = ICDATA_DIR
data_filenames = sorted(glob(data_dir + '/IC86_20*.data.npy'))
sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged.npy'))
grl_filenames = sorted(glob(data_dir + '/GRL/IC86_20*.data.npy'))

################ energy lower bound #############
min_log_e = np.log10(10)
#################################################
bins_sindec = np.linspace(-1, 1, 25+1)  
bins_logenergy = np.linspace(min_log_e, 4, 25+1)

data = [np.load(data_filename) for data_filename in data_filenames]
data = np.hstack(data)
sig = [np.load(sig_filename) for sig_filename in sig_filenames]
sig = np.hstack(sig)
grl = [np.load(grl_filename) for grl_filename in grl_filenames]
grl = np.hstack(grl)
if min_log_e is not None:
    data_mask = data['logE'] > min_log_e
    data = data[data_mask]
    sig_mask = sig['logE'] > min_log_e
    sig = sig[sig_mask]
    
dataset_spec = cy.selections.CustomDataSpecs.CustomDataSpec(data, sig, np.sum(grl['livetime']),
                                                     sindec_bins=bins_sindec,
                                                     logenergy_bins=bins_logenergy,
                                                     grl=grl, key='greco_v2.10', cascades=True)

ANA_DIR = cy.utils.ensure_dir(ANA_DIR)
# on OSG
# ana_dir = "./"
ana = cy.get_analysis(cy.selections.repo
                      , dataset_spec
                      , dir=ANA_DIR
                      , load_sig=True)  # to save memory: use False



===== Setting up csky =====

Setting up Analysis for:
greco_v2.10
Setting up greco_v2.10...
Energy PDF Ratio Model...
  * gamma = 4.0000 ...
Signal Acceptance Model...
  * gamma = 4.0000 ...
Done.


In [4]:
############# used for basic trial_runner
# conf = {
#     'ana': ana,
#     #### llh basics: csky.conf
#     'space': 'prior', # ps/fitps/template/prior
#     'time': 'transient', # utf/lc/transient
#     'energy': 'customflux', # fit/customflux
#     'flux': cy.hyp.PowerLawFlux(2.5),
#     #### inj.py - prior has some duplications against space's prior
#     'sig': 'transient', # ps/tw/lc/transient/template/prior
#     'full_sky': True,
#     'extended': True,
#     'cut_n_sigma': 3,
#     "TRUTH": True,
#     'mp_cpus': args.ncpu
# }

############# used for spatial_prior_trial_runner
conf = {
    'ana': ana,
    #### llh basics: csky.conf
    'space': 'ps', # ps/fitps/template/prior
    'time': 'transient', # utf/lc/transient
    'energy': 'customflux', # fit/customflux
    'flux': cy.hyp.PowerLawFlux(2.5),
    #### inj.py - prior has some duplications against space's prior
    'sig': 'transient', # ps/tw/lc/transient/template/prior
    'full_sky': True,
    'extended': True,
    'mp_cpus': args.ncpu,
    'cut_n_sigma': 3
}

cy.CONF.update(conf)

# save every GRB's:
## for all 6 time windows:
### best_TW, best_TS

In [5]:
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")

In [9]:
# for each grb
for grb_idx in range(args.grb_idx_start+4, args.grb_idx_end):
    # load grb info
    grb_name = df.grb_name[grb_idx]
    print(f"Working on {grb_name}")
    grb_row = df.loc[df['grb_name'] == grb_name]
    ra = grb_row.ra
    dec = grb_row.dec
    # load grb healpix map
    healpix = np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name))
    healpix = np.maximum(healpix,0)
    ########## healpix reduce (< instead of <=) ##########
    healpix[healpix < isf_healpix(healpix, q=0.99)] = 0
    healpix = healpix / np.sum(healpix)
    # for each tw_in_second
    tss, nss = [], []
    for tw_in_second in [10,25,50,100,250,500]:
        tw = tw_in_second/86400.
        tw_start = grb_row.t_center - 0.5*tw
        src = cy.sources(
            ra=ra,
            dec=dec,
            deg=True,
            mjd=tw_start, 
            sigma_t=np.zeros_like(tw), 
            t_100=tw,  # in days
            prior=[hl.heal.HealHist(healpix)],
            name=grb_name
        )
#         tr = cy.get_trial_runner(conf=cy.CONF, ana=ana, src=src)
#         result = ts, ns = tr.get_one_fit(TRUTH=True, logging=False)
        sptr = cy.get_spatial_prior_trial_runner(conf=cy.CONF
                                         ,src_tr=src
                                         ,llh_priors=[healpix])
        
        print(f"tw_in_second = {tw_in_second}")
        try:
            result = _, ts, ns, ra, dec = sptr.get_one_fit(TRUTH=True, mp_cpus=args.ncpu, logging=False)
        except:
            ts, ns = 0.0, 0.0
        tss.append(ts)
        nss.append(ns)
    # get pvals
    bg_files = sorted(glob(ANA_DIR + f"/allsky_scan/with_prior_background/tw*/{grb_name}_*.npz"), 
                 key=lambda x: int(x[x.find("/tw")+3:x.find(f"/{grb_name}")]))
    assert len(bg_files) > 0, print(f"cannot find bkg files of {grb_name}")
    bgs = np.array([sparse.load_npz(bg_file).toarray()[0] for bg_file in bg_files])
    bgs_sorted = np.apply_along_axis(sorted, 1, bgs)
    pvals = []
    for ts, bg_sorted in zip(tss, bgs_sorted):
        pvals.append((bg_sorted.size - np.searchsorted(bg_sorted, ts, side='left')) / bg_sorted.size)
    tss = np.array(tss)
    nss = np.array(nss)
    pvals = np.array(pvals)
    print(tss, nss, pvals)
    try:
        ts_ns_pval = np.array(
            list(map(tuple, np.transpose([tss,nss,pvals]))),
            dtype=np.dtype(
                [
                    ('ts', np.float32), 
                    ('ns', np.float32), 
                    ('pval', np.float32)
                ]
            )
        )
        output_folder = cy.utils.ensure_dir(args.out_dir+f"/unblind/ts_ns_p")
        print(f"Saving {output_folder}/{grb_name}_ts_ns_p.npy")
        print(ts_ns_pval)
        np.save(output_folder + f"/{grb_name}_ts_ns_p.npy", ts_ns_pval)
    except:
        print(f"An exception occurs when saving {grb_name}_ts_ns_p.npy")

Working on GRB190916A
tw_in_second = 10
tw_in_second = 25
tw_in_second = 50
tw_in_second = 100
tw_in_second = 250
tw_in_second = 500
exception raised trying to construct <class 'csky.trial.TrialRunner'>


Process Process-9:
Traceback (most recent call last):
  File "/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/csky/trial.py", line 1921, in get_partial_scan_from_trial
    tr = self.get_tr(utils.Sources(ra=0, dec=sdec), inj=False)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/csky/trial.py", line 1010, in <lambda>
    get_tr = lambda *a, **kw: self.get_tr(*a, ana=self.ana, **kw)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/csky/conf.py", line 351, in get_tr
    return get_trial_runner(conf=conf, inj_conf=inj_conf, src=src, ana=ana, **kw)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/

exception raised trying to construct <class 'csky.trial.TrialRunner'>
exception raised trying to construct <class 'csky.trial.TrialRunner'>


Process Process-12:
Process Process-11:
Process Process-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/csky/trial.py", line 1924, in get_partial_scan_from_trial
    e, n = self.select_from_trial(selector, (evss, ns_excluded))
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky/csky/trial.py", line 1877, in select_from_trial
    new_ev = selector(ev)
  File "/storage/hive/project/phy-taboada/cchen641/icecube/software/csky

KeyboardInterrupt: 